this session is about predicting the stock performance of google. we get the google stock data from quandl. read documentation for better understanding

In [20]:
import pandas as pd
import quandl
import math

In [8]:
#go to quandl.com and search for dataset for google. i've used google's here.
df=quandl.get('WIKI/GOOGL')

In [9]:
df.tail()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-03-21,1092.57,1108.70,1087.21,1094.00,1990515.0,0.0,1.0,1092.57,1108.70,1087.21,1094.00,1990515.0
2018-03-22,1080.01,1083.92,1049.64,1053.15,3418154.0,0.0,1.0,1080.01,1083.92,1049.64,1053.15,3418154.0
2018-03-23,1051.37,1066.78,1024.87,1026.55,2413517.0,0.0,1.0,1051.37,1066.78,1024.87,1026.55,2413517.0
2018-03-26,1050.60,1059.27,1010.58,1054.09,3272409.0,0.0,1.0,1050.60,1059.27,1010.58,1054.09,3272409.0
2018-03-27,1063.90,1064.54,997.62,1006.94,2940957.0,0.0,1.0,1063.90,1064.54,997.62,1006.94,2940957.0


the columns in the df are features open,close,high,low

adj. high and low are adjusted high and low. they are adjusted if a company gave a thought on lets say that our share is currently priced at 1000$ it would be great if our share is 500$ people will buy more actively.

so the company says that now two of our shares are a 1000 dollars now. hence reflected in adj. openfrom the next day.

so we dnt need all the features to predict the stock's performance some of importance we will consider as per needs

__lets grab some features of interset__

In [10]:
df=df[['Adj. Open','Adj. Close','Adj. High','Adj. Low','Adj. Volume']]

In [11]:
df.head()

,Adj. Open,Adj. Close,Adj. High,Adj. Low,Adj. Volume
Date,,,,,
2004-08-19,50.159839,50.322842,52.191109,48.128568,44659000.0
2004-08-20,50.661387,54.322689,54.708881,50.405597,22834300.0
2004-08-23,55.551482,54.869377,56.915693,54.693835,18256100.0
2004-08-24,55.792225,52.597363,55.972783,51.945350,15247300.0
2004-08-25,52.542193,53.164113,54.167209,52.100830,9188600.0


now we have adjusted open-> open price of share, close->close price, low->how low share went, high-> how high share went

we will create a high_low_percent column

In [13]:
#this is the high low percentage ((high-close)/close)x100
df['HighLow_PCT'] = (df['Adj. High']-df['Adj. Close'])/df['Adj. Close'] *100

#daily change ((close-open)/close)x100
df['PCT_DailyChange'] = (df['Adj. Close']-df['Adj. Open'])/df['Adj. Open'] *100

In [14]:
df.head()

,Adj. Open,Adj. Close,Adj. High,Adj. Low,Adj. Volume,HighLow_PCT,PCT_DailyChange
Date,,,,,,,
2004-08-19,50.159839,50.322842,52.191109,48.128568,44659000.0,3.712563,0.324968
2004-08-20,50.661387,54.322689,54.708881,50.405597,22834300.0,0.710922,7.227007
2004-08-23,55.551482,54.869377,56.915693,54.693835,18256100.0,3.729433,-1.227880
2004-08-24,55.792225,52.597363,55.972783,51.945350,15247300.0,6.417469,-5.726357
2004-08-25,52.542193,53.164113,54.167209,52.100830,9188600.0,1.886792,1.183658


now we can get rid of open close high low we have pct_change and highlow pct

In [16]:
#volume represents how much trading happened for the day so that's important too.
df= df[['Adj. Close','PCT_DailyChange','HighLow_PCT','Adj. Volume']]

In [17]:
df.head()

,Adj. Close,PCT_DailyChange,HighLow_PCT,Adj. Volume
Date,,,,
2004-08-19,50.322842,0.324968,3.712563,44659000.0
2004-08-20,54.322689,7.227007,0.710922,22834300.0
2004-08-23,54.869377,-1.227880,3.729433,18256100.0
2004-08-24,52.597363,-5.726357,6.417469,15247300.0
2004-08-25,53.164113,1.183658,1.886792,9188600.0


In [25]:
forecast_col='Adj. Close'

In [26]:
#fill any missing value in dataframe i've chosen -99999 here
df.fillna(-99999,inplace=True)

In [27]:
forecast_out=int(math.ceil(0.01*len(df)))
df['label']=df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)

In [29]:
df.head()

,Adj. Close,PCT_DailyChange,HighLow_PCT,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,0.324968,3.712563,44659000.0,69.078238
2004-08-20,54.322689,7.227007,0.710922,22834300.0,67.839414
2004-08-23,54.869377,-1.227880,3.729433,18256100.0,68.912727
2004-08-24,52.597363,-5.726357,6.417469,15247300.0,70.668146
2004-08-25,53.164113,1.183658,1.886792,9188600.0,71.219849


__let's compute now__

In [32]:
import numpy as np
from sklearn import preprocessing,svm,cross_validation
from sklearn.linear_model import LinearRegression

In [33]:
X= np.array(df.drop(['label'],1))

In [34]:
y= np.array(df['label'])

In [35]:
X=preprocessing.scale(X) #scaling X

In [38]:
len(X)

3389

In [39]:
len(y)

3389

In [41]:
X_train,X_test,y_train,y_test=cross_validation.train_test_split(X,y,test_size=0.2)

In [42]:
#classifier
clf =LinearRegression()

#fit is to train
clf.fit(X_train,y_train)

#score is to test
accuracy=clf.score(X_test,y_test)


In [43]:
accuracy

0.9773568883237574

In [44]:
forecast_out

35